In [10]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import json
import os
import datetime as dt
from tqdm import tqdm
import pandas as pd
import numpy as np

In [11]:
def f2cat(filename: str) -> str:
    return filename.split('.')[0]

class Simplified():
    def __init__(self, input_path='dataset/train_simplified/'):
        self.input_path = input_path

    def list_all_categories(self):
        #files = os.listdir(os.path.join(self.input_path, 'train_simplified'))
        files = os.listdir(os.path.join(self.input_path))
        return sorted([f2cat(f) for f in files], key=str.lower)

    def read_training_csv(self, category, nrows=None, usecols=None, drawing_transform=False):
        df = pd.read_csv(os.path.join(self.input_path, category + '.csv'),
                         nrows=nrows, parse_dates=['timestamp'], usecols=usecols)
        if drawing_transform:
            df['drawing'] = df['drawing'].apply(json.loads)
        return df
    



In [12]:
start = dt.datetime.now()
CSV_DIR = "dataset/train_simplified/"
s = Simplified(CSV_DIR)
NCSVS = 100
categories = s.list_all_categories()
print("Total categories: {}".format(len(categories)))

Total categories: 340


In [13]:
DIR_PATH = 'dataset/train_simplified_all_shuffle' # The dir of processed suffle dataset

for y, cat in tqdm(enumerate(categories)):
    #df = s.read_training_csv(cat, nrows=30000)
    df = s.read_training_csv(cat, nrows=None)
    df['y'] = y
    df['cv'] = (df.key_id // 10 ** 7) % NCSVS
    for k in range(NCSVS):
        filename = 'train_k{}.csv'.format(k)
        chunk = df[df.cv == k]
        chunk = chunk.drop(['key_id'], axis=1)
        if y == 0:
            chunk.to_csv(os.path.join(DIR_PATH, filename), index=False)
        else:
            chunk.to_csv(os.path.join(DIR_PATH, filename), mode='a', header=False, index=False)

340it [12:59,  2.29s/it]


In [14]:
for k in tqdm(range(NCSVS)):
    filename = 'train_k{}.csv'.format(k)
    filename = os.path.join(DIR_PATH, filename)
    if os.path.exists(filename):
        df = pd.read_csv(filename)
        df['rnd'] = np.random.rand(len(df))
        df = df.sort_values(by='rnd').drop('rnd', axis=1)
        df.to_csv(filename + '.gz', compression='gzip', index=False)
        os.remove(filename)
print(df.shape)

100%|██████████| 100/100 [3:20:38<00:00, 120.39s/it] 

(497740, 7)
